# Herramientas para Procesamiento de Lenguaje Natural (PLN)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('plnHerr').getOrCreate()

### Tokenización

Tokenización es el proceso de tomar un texto y dividirlo en terminos individuales (usualmente palabras), lo convierte en una secuencia de palabras.
- La clase Tokenizer permite la tokenización
- La clase RegexTokenizer permite hacer la tokenización usando expresiones regulares

In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [0]:
df_documentos = spark.createDataFrame([
    (0,"Hola esta es una prueba con Spark"),
    (1,"Java está incluida para su uso en Spark"),
    (2,"Sigmoide,es,una,función,de,activación")
],["id","oracion"])

In [0]:
df_documentos.show(truncate=False)

+---+---------------------------------------+
|id |oracion                                |
+---+---------------------------------------+
|0  |Hola esta es una prueba con Spark      |
|1  |Java está incluida para su uso en Spark|
|2  |Sigmoide,es,una,función,de,activación  |
+---+---------------------------------------+



In [0]:
tokenizador = Tokenizer(inputCol="oracion",outputCol="palabras")

In [0]:
tokenizadorER = RegexTokenizer(inputCol="oracion",outputCol="palabras",pattern="\\W")

In [0]:
contarTokens = udf(lambda palabras: len(palabras), IntegerType())

In [0]:
tokenizado = tokenizador.transform(df_documentos)

In [0]:
tokenizado.show(truncate=False)

+---+---------------------------------------+------------------------------------------------+
|id |oracion                                |palabras                                        |
+---+---------------------------------------+------------------------------------------------+
|0  |Hola esta es una prueba con Spark      |[hola, esta, es, una, prueba, con, spark]       |
|1  |Java está incluida para su uso en Spark|[java, está, incluida, para, su, uso, en, spark]|
|2  |Sigmoide,es,una,función,de,activación  |[sigmoide,es,una,función,de,activación]         |
+---+---------------------------------------+------------------------------------------------+



In [0]:
tokenizado.select("oracion","palabras")\
    .withColumn("tokens",contarTokens(col("palabras"))).show(truncate=False)
    

+---------------------------------------+------------------------------------------------+------+
|oracion                                |palabras                                        |tokens|
+---------------------------------------+------------------------------------------------+------+
|Hola esta es una prueba con Spark      |[hola, esta, es, una, prueba, con, spark]       |7     |
|Java está incluida para su uso en Spark|[java, está, incluida, para, su, uso, en, spark]|8     |
|Sigmoide,es,una,función,de,activación  |[sigmoide,es,una,función,de,activación]         |1     |
+---------------------------------------+------------------------------------------------+------+



In [0]:
tokenizadoER = tokenizadorER.transform(df_documentos)

In [0]:
tokenizadoER.select("oracion","palabras")\
    .withColumn("tokens",contarTokens(col("palabras"))).show(truncate=False)

+---------------------------------------+-----------------------------------------------+------+
|oracion                                |palabras                                       |tokens|
+---------------------------------------+-----------------------------------------------+------+
|Hola esta es una prueba con Spark      |[hola, esta, es, una, prueba, con, spark]      |7     |
|Java está incluida para su uso en Spark|[java, est, incluida, para, su, uso, en, spark]|8     |
|Sigmoide,es,una,función,de,activación  |[sigmoide, es, una, funci, n, de, activaci, n] |8     |
+---------------------------------------+-----------------------------------------------+------+



### Eliminación de Stop Words

**Stop words** son palabras que serán excluidas de la entrada, típicamente porque estas palabras aparecen frecuentemente y no aportan mucho significado al análisis.

Se usaran la clase *StopWordsRemover*, está cñase toma una secuencia de cadenas y elimina los stop words de esta secuencia. *StopWordsRemover.loadDefaultWords(language)* permite acceder a los Stop Words por defecto en un idioma (Por ejemplo: "spanish"). La clase también tiene un parametro booleano *caseSensitive* para configurar si es sensible a mayúsculas y minúsculas (por defecto False)

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
datosPalabra = spark.createDataFrame([
    (0,["I","saw","the","red","balloon"]),
    (1,["Mary","had","a","little","lamb"])
],["id","palabras"])

In [0]:
datosPalabra.show(truncate=False)

+---+----------------------------+
|id |palabras                    |
+---+----------------------------+
|0  |[I, saw, the, red, balloon] |
|1  |[Mary, had, a, little, lamb]|
+---+----------------------------+



In [0]:
# Crea el filtro
filtro = StopWordsRemover(inputCol="palabras",outputCol="filtrado")

In [0]:
filtro.transform(datosPalabra).show(truncate=False)

+---+----------------------------+--------------------+
|id |palabras                    |filtrado            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



In [0]:
datosPalabra2 = spark.createDataFrame([
    (0,["Yo","vi","el","balón","rojo"]),
    (1,["Mary","tiene","un","pequeño","cordero"])
],["id","palabras"])

In [0]:
datosPalabra2.show(truncate=False)

+---+-----------------------------------+
|id |palabras                           |
+---+-----------------------------------+
|0  |[Yo, vi, el, balón, rojo]          |
|1  |[Mary, tiene, un, pequeño, cordero]|
+---+-----------------------------------+



In [0]:
filtroEspa = StopWordsRemover(inputCol="palabras",outputCol="filtrado")

In [0]:
filtroEspa.loadDefaultStopWords('spanish')
filtroEspa.transform(datosPalabra2).show(truncate=False)

+---+-----------------------------------+-----------------------------------+
|id |palabras                           |filtrado                           |
+---+-----------------------------------+-----------------------------------+
|0  |[Yo, vi, el, balón, rojo]          |[Yo, vi, el, balón, rojo]          |
|1  |[Mary, tiene, un, pequeño, cordero]|[Mary, tiene, un, pequeño, cordero]|
+---+-----------------------------------+-----------------------------------+



In [0]:
!pip install nltk

     |████████████████████████████████| 1.5 MB 5.8 MB/s 
     |████████████████████████████████| 96 kB 8.6 MB/s 
     |████████████████████████████████| 772 kB 37.0 MB/s 
     |████████████████████████████████| 78 kB 5.5 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Out[32]: True

In [0]:
stopwordList = nltk.corpus.stopwords.words('spanish')

In [0]:
filtroEspa2 = StopWordsRemover(inputCol="palabras",outputCol="filtrado",stopWords=stopwordList)

In [0]:
filtroEspa2.transform(datosPalabra2).show(truncate=False)

+---+-----------------------------------+------------------------+
|id |palabras                           |filtrado                |
+---+-----------------------------------+------------------------+
|0  |[Yo, vi, el, balón, rojo]          |[vi, balón, rojo]       |
|1  |[Mary, tiene, un, pequeño, cordero]|[Mary, pequeño, cordero]|
+---+-----------------------------------+------------------------+



### n grama

Un n-grama es una secuencia de nn tokens (tipicamente palabras) para algún entero n. Usaremos la clase NGram para transformar las entradas en nn-gramas.

In [0]:
from pyspark.ml.feature import NGram

In [0]:
df_palabras = spark.createDataFrame([
    (0,["Hola","esta","es","una","prueba","con","Spark"]),
    (1,["Java","está","incluido","para","su","uso","en","Spark"]),
    (2,["Sigmoide","es","una","función","de","activación"])
],["id","palabras"])

In [0]:
df_palabras.show(truncate=False)

+---+------------------------------------------------+
|id |palabras                                        |
+---+------------------------------------------------+
|0  |[Hola, esta, es, una, prueba, con, Spark]       |
|1  |[Java, está, incluido, para, su, uso, en, Spark]|
|2  |[Sigmoide, es, una, función, de, activación]    |
+---+------------------------------------------------+



In [0]:
ngrama = NGram(n=2,inputCol='palabras',outputCol="ngramas")

In [0]:
ngramaDF = ngrama.transform(df_palabras)

In [0]:
ngramaDF.select("ngramas").show(truncate=False)

+----------------------------------------------------------------------------+
|ngramas                                                                     |
+----------------------------------------------------------------------------+
|[Hola esta, esta es, es una, una prueba, prueba con, con Spark]             |
|[Java está, está incluido, incluido para, para su, su uso, uso en, en Spark]|
|[Sigmoide es, es una, una función, función de, de activación]               |
+----------------------------------------------------------------------------+

